In [1]:
from data import *
from pretrained_models import *
from torch.utils.data import DataLoader

In [2]:
MediaPipeCFG = MediaPipeCfg("pretrained_model/hand_landmarker.task")
options = MediaPipeCFG.create_options()
MP_model = MediaPipeCFG.HandLandmarker.create_from_options(options)

In [3]:
MMPoseCFG = MMPoseCfg(checkpoint_path='pretrained_model/checkpoint/rtmpose-s_simcc-body7_pt-body7_420e-256x192-acd4a1ef_20230504.pth',
                      config_path='pretrained_model/mmpose_config/rtmpose_m_8xb256-420e_coco-256x192.py')
body_model = MMPoseCFG.create_model()

Loads checkpoint by local backend from path: pretrained_model/checkpoint/rtmpose-s_simcc-body7_pt-body7_420e-256x192-acd4a1ef_20230504.pth
The model and loaded state dict do not match exactly

size mismatch for backbone.stem.0.conv.weight: copying a param with shape torch.Size([16, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([24, 3, 3, 3]).
size mismatch for backbone.stem.0.bn.weight: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([24]).
size mismatch for backbone.stem.0.bn.bias: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([24]).
size mismatch for backbone.stem.0.bn.running_mean: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([24]).
size mismatch for backbone.stem.0.bn.running_var: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([24]).
size mis

In [4]:
PreTrainDataset = How2Sign(video_dir="data/raw_videos",
                           MP_model=MP_model,
                           body_cfg=MMPoseCFG,
                           body_model=body_model,
                           labels_path="data/how2sign_realigned_train.csv")
def collate_as_is(batch):
    return batch
PreTrainDataLoader = DataLoader(PreTrainDataset, batch_size=4, shuffle=False, collate_fn=collate_as_is)

In [5]:
OneSample = next(iter(PreTrainDataLoader))

### Test Conversion To JSON Code

In [14]:
OneSample[0]['hand_landmarks']

{'num_frames': 365,
 'frames': [{'frame_index': 0,
   'result': {'hand_landmarks': [[{'x': 0.5668863654136658,
       'y': 0.5799674987792969,
       'z': -8.213499214093645e-09},
      {'x': 0.5825051069259644,
       'y': 0.5503950715065002,
       'z': 0.0006479280418716371},
      {'x': 0.5921515226364136,
       'y': 0.5150333046913147,
       'z': 0.00015542958863079548},
      {'x': 0.5979565978050232,
       'y': 0.4886307120323181,
       'z': -0.0013665941078215837},
      {'x': 0.6030817627906799,
       'y': 0.4685939848423004,
       'z': -0.0032976141665130854},
      {'x': 0.5914514660835266,
       'y': 0.5021907687187195,
       'z': -0.0014923162525519729},
      {'x': 0.5947654247283936,
       'y': 0.4662100374698639,
       'z': -0.007608477491885424},
      {'x': 0.5986630916595459,
       'y': 0.4459690451622009,
       'z': -0.01261178683489561},
      {'x': 0.6038383841514587,
       'y': 0.42984646558761597,
       'z': -0.016453519463539124},
      {'x': 0.58